In [1]:
import os
import requests
from shapely.geometry import shape, Polygon, MultiPolygon
from shapely.ops import unary_union
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta

# Set the environment variable to use Shapely instead of PyGEOS
os.environ['USE_PYGEOS'] = '0'

# Define constants
API_URL = 'https://mesonet.agron.iastate.edu/api/1/nws/spc_outlook.geojson'
CYCLE = 16
THRESHOLD_LEVELS = ['MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH']

# Load NYS boundary shapefile and reproject if necessary
nys_boundary_path = '/nfs/home11/ugrad/2020/tr588861/SWRCC/State_Shapefiles/state/State.shp'
zip_population_path = '/nfs/home11/ugrad/2020/tr588861/SWRCC/State_Shapefiles/zip_codes/zip_population.shp'

nys_boundary = gpd.read_file(nys_boundary_path)
nys_boundary.to_crs(epsg=4326, inplace=True)  # Reproject to WGS84 if needed
nys_boundary_union = unary_union(nys_boundary.geometry)

zip_population = gpd.read_file(zip_population_path)

# Calculate the total area and population of New York State
total_area = nys_boundary_union.area * (111 ** 2)  # Total area in square kilometers
total_population = zip_population['Population'].sum()

# Function to fetch and process data for the given date
def fetch_and_process_data(date):
    params = {
        'day': 1,
        'valid': date.strftime('%Y-%m-%dT00:00'),
        'cycle': CYCLE,
        'outlook_type': 'C'
    }
    
    response = requests.get(API_URL, params=params)
    data = response.json()
    
    # Dictionary to track the highest threshold level for each zip code
    zip_highest_threshold = {}
    
    # Dictionary to store the area sum under each threshold level
    threshold_areas = {level: 0 for level in THRESHOLD_LEVELS}
    threshold_polygons = {level: [] for level in THRESHOLD_LEVELS}

    for feature in data['features']:
        properties = feature['properties']
        geometry = feature['geometry']
        
        if properties['category'] == 'CATEGORICAL':
            threshold = properties['threshold']
            
            if threshold not in THRESHOLD_LEVELS:
                continue

            # Convert the geometry to a Shapely object (handling MultiPolygon)
            polygons = shape(geometry)
            if isinstance(polygons, Polygon):
                polygons = [polygons]
            elif isinstance(polygons, MultiPolygon):
                polygons = list(polygons.geoms)

            for polygon in polygons:
                if polygon.is_valid and polygon.intersects(nys_boundary_union):
                    # Calculate the intersection with NYS boundary
                    intersection = polygon.intersection(nys_boundary_union)
                    
                    if intersection.is_empty:
                        continue
                    
                    # Add intersection polygon to the list for the current threshold
                    threshold_polygons[threshold].append(intersection)
                    
                    # Check which zip codes intersect with the intersection polygon
                    for idx, zip_geom in zip_population.iterrows():
                        if zip_geom.geometry.intersects(intersection):
                            zip_code = zip_geom['ZCTA5CE10']  # Assuming the column name for zip code
                            current_level_index = THRESHOLD_LEVELS.index(threshold)
                            
                            # Update the highest threshold level for this zip code
                            if zip_code in zip_highest_threshold:
                                previous_level_index = THRESHOLD_LEVELS.index(zip_highest_threshold[zip_code])
                                if current_level_index < previous_level_index:
                                    zip_highest_threshold[zip_code] = threshold
                            else:
                                zip_highest_threshold[zip_code] = threshold

    # Combine polygons for each threshold level and calculate areas
    for level in THRESHOLD_LEVELS:
        if threshold_polygons[level]:
            combined_polygons = unary_union(threshold_polygons[level])
            threshold_areas[level] = combined_polygons.area * (111 ** 2)  # Area in square kilometers
    
    # Initialize a dictionary to store the population sum under each threshold level
    population_sum = {level: 0 for level in THRESHOLD_LEVELS}

    # Sum the population based on the highest threshold level assigned
    for zip_code, highest_level in zip_highest_threshold.items():
        highest_index = THRESHOLD_LEVELS.index(highest_level)
        population = zip_population.loc[zip_population['ZCTA5CE10'] == zip_code, 'Population'].sum()
        for level in THRESHOLD_LEVELS[:highest_index + 1]:
            population_sum[level] += population
    
    # Aggregate area to include higher levels
    for i in range(len(THRESHOLD_LEVELS) - 1, 0, -1):
        threshold_areas[THRESHOLD_LEVELS[i - 1]] += threshold_areas[THRESHOLD_LEVELS[i]]

    return threshold_areas, population_sum, data

# Set the date range
start_date = datetime(2015, 1, 1)
end_date = datetime.today()

# Create the output directory
output_dir = 'spc_outlook_data'
os.makedirs(output_dir, exist_ok=True)

# Initialize dictionaries to accumulate data for each hazard level
all_data = {level: [] for level in THRESHOLD_LEVELS}

# Loop over each date in the date range
current_date = start_date
while current_date <= end_date:
    #print(f"Processing data for {current_date.strftime('%Y-%m-%d')}...")
    threshold_areas, population_data, data = fetch_and_process_data(current_date)

    # Calculate percentages
    area_percentages = {level: (threshold_areas[level] / total_area) * 100 for level in THRESHOLD_LEVELS}
    population_percentages = {level: (population_data[level] / total_population) * 100 for level in THRESHOLD_LEVELS}

    # Combine area, population, and percentage data into a single table
    for level in THRESHOLD_LEVELS:
        if threshold_areas[level] > 0 and population_data[level] > 0:
            all_data[level].append([
                level,
                current_date.strftime('%Y-%m-%d'),
                threshold_areas[level],
                area_percentages[level],
                population_data[level],
                population_percentages[level]
            ])

    current_date += timedelta(days=1)

# Write the accumulated data to CSV files
for level in THRESHOLD_LEVELS:
    if all_data[level]:  # Only write the CSV if there is data
        df = pd.DataFrame(all_data[level], columns=[
            'Risk Level', 'Date', 'Area (Sq. Miles)', 'Area Percentage (%)', 'Population', 'Population Percentage (%)'
        ])
        filepath = os.path.join(output_dir, f'{level}_data.csv')
        df.to_csv(filepath, index=False)

print("Processing complete.")

/tmp/ipykernel_3932799/167432472.py:5: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


Processing complete.
